- This is for the competition.

In [1]:

import warnings
warnings.simplefilter(action='ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
import sys
import multiprocessing
parent_dir = os.path.join(os.getcwd(), '..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

'''Custom packages'''
from ttknn import ttknn
from ttknn.light_utility import Utility, write_unseen_rows
import geobleu
CHUNK_SIZE = 10_000
USERS_RANGE = (0, 3_000)

ImportError: cannot import name 'write_unseen_rows' from 'ttknn.light_utility' (c:\Users\ss348\anaconda3\Lib\site-packages\ttknn\light_utility.py)

# Data Loading

In [3]:
tar_fold = '../cityB-dataset'
tar_path_list = []
for file in os.listdir(tar_fold):
    curr_dir = os.path.join(os.getcwd(), tar_fold)
    if file.endswith('.csv'):
        tar_path_list.append(os.path.join(curr_dir, file))

In [4]:
df = pd.read_csv(tar_path_list[0])

# Data Filtering

# Collect Predictions

In [17]:
FILE_PATH = '../result/file.csv'
STEPS = 5
all_uids = df.uid.unique()
processing_uids = all_uids[:3000]
train_df, test_df = Utility.test_train_split(df, 60)

num_cpus = multiprocessing.cpu_count()
results = []
if os.path.exists(FILE_PATH):
    curr_df = pd.read_csv(FILE_PATH)
    curr_uids = curr_df['uid'].unique()
    processing_uids  = set(all_uids).difference(curr_uids)
    
for i, uid in enumerate(tqdm(processing_uids)):
    sample = train_df[train_df.uid == uid]
    test_sample = test_df[test_df.uid == uid]

    submission_df = test_sample[['d', 't']]
    predicted = ttknn.temporal_knn_fit_predict(df=sample)
    s1 = Utility.to_eval_format(submission_df.merge(predicted, on='t', how='left'))
    s2 = Utility.to_eval_format(test_sample)

    results.append({"uid": uid, "s1": str(s1), "s2": str(s2)})
    
    # Save intermediate results every n steps
    if (i + 1) % STEPS == 0:
        intermediate_df = pd.DataFrame(results)
        write_unseen_rows(intermediate_df, FILE_PATH)


  1%|          | 191/24958 [01:55<4:08:55,  1.66it/s]


KeyboardInterrupt: 